In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('C:\\Users\\hemanth kumar\\Desktop\\A-Z data sets\\P14-Part2-Regression (1)\\P14-Part2-Regression\\Section 7 - Multiple Linear Regression\\Python\\50_Startups.csv')

In [4]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [5]:
df=round(df,0)

In [6]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.0,136898.0,471784.0,New York,192262.0
1,162598.0,151378.0,443899.0,California,191792.0
2,153442.0,101146.0,407935.0,Florida,191050.0
3,144372.0,118672.0,383200.0,New York,182902.0
4,142107.0,91392.0,366168.0,Florida,166188.0


In [21]:
x=df.iloc[:,0:4]
y=df.iloc[:,4]

In [22]:
df['State'].unique()

array(['New York', 'California', 'Florida'], dtype=object)

In [23]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
p=ohe.fit_transform(x[['State']]).toarray()
p=pd.DataFrame(p)
x=p.merge(x,how='outer',left_index=True,right_index=True).drop('State',axis=1)
x.head()

,0,1,2,R&D Spend,Administration,Marketing Spend
0,0.0,0.0,1.0,165349.0,136898.0,471784.0
1,1.0,0.0,0.0,162598.0,151378.0,443899.0
2,0.0,1.0,0.0,153442.0,101146.0,407935.0
3,0.0,0.0,1.0,144372.0,118672.0,383200.0
4,0.0,1.0,0.0,142107.0,91392.0,366168.0


## Avoding the Dummy variable trap
### written in the notes(if we have n dummy variables to encode use n-1 variables only as encoded)

In [25]:
x=x.drop(0,axis=1)

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.2,random_state=0)

,1,2,R&D Spend,Administration,Marketing Spend
36,1.0,0.0,28664.0,127056.0,201127.0
21,0.0,1.0,78389.0,153773.0,299737.0
19,0.0,1.0,86420.0,153514.0,0.0
9,0.0,0.0,123335.0,108679.0,304982.0
39,0.0,0.0,38559.0,82982.0,174999.0
46,1.0,0.0,1315.0,115816.0,297114.0
3,0.0,1.0,144372.0,118672.0,383200.0
0,0.0,1.0,165349.0,136898.0,471784.0
47,0.0,0.0,0.0,135427.0,0.0
44,0.0,0.0,22178.0,154806.0,28335.0


## linear regression is has built in feature scalling so no need of feature scaling

In [28]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
y_pred=lr.predict(x_test)

# backward elimination preperation

## to apply a stats model we need to add the bias unit

In [42]:
sl=0.05  ## the significance value

### note: it is very important to add the bias unit

In [36]:
o=np.ones((50,1)).astype(int)   ##this code explination is in the vedio
o=pd.DataFrame(o,columns=['bias'])

x=o.merge(x,how='outer',left_index=True,right_index=True)
x.head()

,bias,1,2,R&D Spend,Administration,Marketing Spend
0,1,0.0,1.0,165349.0,136898.0,471784.0
1,1,0.0,0.0,162598.0,151378.0,443899.0
2,1,1.0,0.0,153442.0,101146.0,407935.0
3,1,0.0,1.0,144372.0,118672.0,383200.0
4,1,1.0,0.0,142107.0,91392.0,366168.0


In [37]:
x.columns

Index(['bias', 1, 2, 'R&D Spend', 'Administration', 'Marketing Spend'], dtype='object')

### building model with BACKWARD PROPEGATION

In [41]:
import statsmodels.api as sm ## sm is a library
x_opt=x.loc[:,['bias', 1, 2, 'R&D Spend', 'Administration', 'Marketing Spend']] #considering all possible features
regressor_ols=sm.OLS(endog=y,exog=x_opt).fit()  ## creating regreesor object of optimal least square(OLS) class and
#fitting model with  all possible features
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           1.34e-27
Time:                        14:55:52   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
bias             5.013e+04   6884.809      7.281      0.000    3.62e+04     6.4e+04
1                 198.9502   3371.004      0.059      0.953   -6594.862    6992.763
2                 -41.7832   3256.037     -0.013      0.990   -6603.894    6520.328
R&D Spend           0.8060      0.046     17.369      0.000       0.712       0.900
Administration     -0.0270      0.052     -0.517      0.608      -0.132       0.078
Marketing Spend     0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.783   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.268
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### by this point x_opt is initialised now we need to update x_opt values. this is step-2 done.

## NOTE: update for the stats model library (import statsmodels.api as sm) in the A-Z class old syntax is not working.

### from the above table we are interested in the p-values for implementing step-3 so as step-4 and step doing all at once here.

In [43]:
x_opt=x.loc[:,['bias', 1, 'R&D Spend', 'Administration', 'Marketing Spend']] #here we removed column '2'
regressor_ols=sm.OLS(endog=y,exog=x_opt).fit()  ## creating regreesor object of optimal least square(OLS) class and
#fitting model with  all possible features
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           8.49e-29
Time:                        15:13:16   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
bias             5.011e+04   6647.857      7.537      0.000    3.67e+04    6.35e+04
1                 220.2669   2900.534      0.076      0.940   -5621.708    6062.241
R&D Spend           0.8060      0.046     17.606      0.000       0.714       0.898
Administration     -0.0270      0.052     -0.523      0.604      -0.131       0.077
Marketing Spend     0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.759   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.174
Skew:                          -0.948   Prob(JB):                     2.52e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### repeating and moving on

In [44]:
x_opt=x.loc[:,['bias', 'R&D Spend', 'Administration', 'Marketing Spend']] #removing column-1
regressor_ols=sm.OLS(endog=y,exog=x_opt).fit()  ## creating regreesor object of optimal least square(OLS) class and
#fitting model with  all possible features
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           4.53e-30
Time:                        15:14:41   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
bias             5.012e+04   6572.339      7.626      0.000    3.69e+04    6.34e+04
R&D Spend           0.8057      0.045     17.846      0.000       0.715       0.897
Administration     -0.0268      0.051     -0.525      0.602      -0.130       0.076
Marketing Spend     0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.839   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.444
Skew:                          -0.949   Prob(JB):                     2.20e-05
Kurtosis:                       5.587   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [45]:
x_opt=x.loc[:,['bias', 'R&D Spend', 'Marketing Spend']] #removing column administration
regressor_ols=sm.OLS(endog=y,exog=x_opt).fit()  ## creating regreesor object of optimal least square(OLS) class and
#fitting model with  all possible features
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           2.15e-31
Time:                        15:15:27   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
bias             4.698e+04   2689.930     17.464      0.000    4.16e+04    5.24e+04
R&D Spend           0.7966      0.041     19.266      0.000       0.713       0.880
Marketing Spend     0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.678   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.163
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.576   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## since our significance level=0.05, so in actual we need to remove the marketing spend, but in this case if we consider the adjusted r^2 values of all the steps, it will tell us it is good to keep marketing spend feature. see video:evaluating regressionmodel performance.
## r^2 error==cost function and adjusted r^2==regularized cost function(andrew n.g)

In [46]:
x_opt=x.loc[:,['bias','R&D Spend']] #removing marketing spend column
regressor_ols=sm.OLS(endog=y,exog=x_opt).fit()  ## creating regreesor object of optimal least square(OLS) class and
#fitting model with  all possible features
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Fri, 03 Jul 2020   Prob (F-statistic):           3.50e-32
Time:                        15:18:14   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
bias        4.903e+04   2537.890     19.320      0.000    4.39e+04    5.41e+04
R&D Spend      0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.728   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.538
Skew:                          -0.911   Prob(JB):                     9.43e-05
Kurtosis:                       5.362   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## conclusion: the R&D spend power full parameter in determinig the profit.